# Encoder-only architecture - BERT 

## 1. Introduction

This notebook will walk through the basics of the BERT architecture, how its tokenizer works, and how positional encoding is applied. 

BERT (Bidirectional Encoder Representations from Transformers) is a transformer-based model designed for NLP tasks. It has multiple layers of encoders, each containing:
- Self-attention mechanism
- Feed-forward neural networks
- Layer normalization and residual connections

We'll start by visualizing the structure of BERT's encoders.

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import matplotlib.pyplot as plt

# Load pre-trained BERT model
model = BertModel.from_pretrained('bert-base-uncased')
print(model)

### 1. **Embeddings Layer**
The embeddings layer is responsible for creating representations of the input tokens. It includes:

- **Word Embeddings**: Maps each of the 30,522 vocabulary tokens to a 768-dimensional vector.
- **Position Embeddings**: Adds positional information to each token using a 512-length sequence of 768-dimensional vectors.
- **Token Type Embeddings**: Adds embeddings to distinguish between segments (e.g., Sentence A and Sentence B) with 2 different token types.
- **Layer Normalization and Dropout**: Normalizes the embeddings and applies dropout to prevent overfitting.

### 2. **BERT Encoder**
BERT's encoder is composed of 12 identical layers (for BERT-base, 24 for BERT-large), each called a **BERT Layer**. Each BERT layer consists of the following components:

- **Self-Attention**:
  - The self-attention mechanism is divided into multiple heads, each computing its own attention scores using three linear projections: Query (`Q`), Key (`K`), and Value (`V`). 

- **Self-Attention Output**:
  - The output of the self-attention mechanism is passed through a dense (linear) layer that projects the output back to the original dimension of 768.

- **Feed-Forward Neural Network**:
  - Each BERT layer includes a position-wise feed-forward network that processes each token independently after the self-attention mechanism.
  - The feed-forward network consists of two dense layers:
    - The **intermediate layer** expands the dimensionality from 768 to 3072 using a linear transformation and applies a **GELU** (Gaussian Error Linear Unit) activation function for non-linearity.
    - The **output layer** projects the 3072-dimensional vector back down to 768 dimensions, matching the input size.

- **Residual Connections**:
  - BERT uses residual (skip) connections around the self-attention and feed-forward components. The input to each sub-layer is added to its output. 

### **BERT Tokenization with WordPiece**

BERT uses a **WordPiece tokenizer**, a subword tokenization technique. This tokenizer splits words into smaller subwords or tokens, allowing BERT to represent rare words as combinations of common subwords and prefixes.

#### **How WordPiece Works**

The WordPiece tokenizer splits a word based on its frequency in the training data:
- **Common Words**: If a word is frequently used in the training corpus, it will likely be stored as a single token. For example, words like "hello" or "sentence" are common enough to exist in BERT's vocabulary as single tokens.
- **Rare or Unknown Words**: For less common or out-of-vocabulary words, the tokenizer will break them into subword units or prefixes. These subwords are often combined with a special prefix `##` to indicate they are part of a larger word. 

Let’s see how the WordPiece tokenizer processes a sample sentence using BERT’s tokenizer.


In [ ]:
# Load pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentence = "hello, this is a sentence!"

# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print("Tokens:", tokens)

# Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

In [ ]:
sentence = "This sentence is too cumbersome!"

# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print("Tokens:", tokens)

# Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

### Special Tokens in BERT

BERT uses several special tokens to structure the input and help the model understand the context and relationships between sequences. These tokens play an important role in various NLP tasks such as sentence classification, next sentence prediction, and more.

#### 1. **[CLS] Token** (Classification Token)
- The `[CLS]` token is added at the beginning of every input sequence. 
- BERT uses this token as a representation of the entire sequence. The hidden state of the `[CLS]` token after passing through BERT can be used for classification tasks (we are going to use it later!).

#### 2. **[PAD] Token** (Padding Token)
- The `[PAD]` token is used to pad sequences so they all have the same length within a batch. The attention mechanism then ignores these `[PAD]` tokens.

In [ ]:
sentence1 = "This is a short sentence."
tokens_single = tokenizer(sentence1, return_tensors='pt', padding='max_length', max_length=10)

# Display tokenized output and input IDs
print("Single Sentence Tokenization:")
print("Tokens:", tokenizer.convert_ids_to_tokens(tokens_single['input_ids'][0]))
print("Input IDs:", tokens_single['input_ids'][0])

#### 3. **[SEP] Token** (Separator Token)
- The `[SEP]` token is used to separate segments (sentences) in BERT's input.
- It is especially important for tasks like Next Sentence Prediction (NSP) where BERT needs to understand the relationship between two sentences.
- It is also added at the end of the sequence to indicate its termination.

In [ ]:
sentence2 = "But now it is a little longer."
tokens_pair = tokenizer(sentence1, sentence2, return_tensors='pt', padding='max_length', max_length=20)

# Display tokenized output and input IDs
print("\nTwo Sentences Tokenization:")
print("Tokens:", tokenizer.convert_ids_to_tokens(tokens_pair['input_ids'][0]))
print("Input IDs:", tokens_pair['input_ids'][0])

#### 4. **[MASK] Token**
- The `[MASK]` token is used during training for the Masked Language Modeling (MLM) task, where a percentage of tokens are masked and the model must predict them based on the context.
- This token is rarely used during inference but is crucial in pre-training BERT.

In [ ]:
# show a mask token 
mask_token = tokenizer.mask_token 
mask_token_id = tokenizer.convert_tokens_to_ids(mask_token)

# Encode a text
text = "I do not know how the sentence will end."
encoded_input = tokenizer(text, return_tensors='pt')

mask_position = 9
encoded_input['input_ids'][0, mask_position] = mask_token_id

print("Tokens:", tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0]))

### Positional Encodings in Transformers

There are two common approaches for implementing positional encodings:

#### 1. **Static Sinusoidal Positional Encodings**

- The one that we already saw in the previous laboratory. 
- **Advantages**:
  - This method requires no additional parameters or learning during training.
  - The same encoding can be applied across sequences of any length without adjustment.
- **Disadvantages**:
  - Since these encodings are static, they might not be as flexible or as specific as learned positional embeddings, which can adapt better to the data.

#### 2. **Learned Positional Embeddings**

- Instead of using a fixed, pre-defined encoding, learned positional embeddings are trained as part of the model, similar to how word embeddings are learned.
- The model learns a set of positional vectors, where each vector corresponds to a position in the sequence (e.g., position 0, position 1, etc.). 
- **Advantages**:
  - The model can change positional encodings to the data it is trained on, potentially capturing positional relationships more effectively.
- **Disadvantages**:
  - Learned positional embeddings introduce additional parameters, which increases the model size and training complexity.
  - The learned embeddings are limited to a maximum sequence length (e.g., 512 for BERT).

BERT uses the second approach: **learned positional embeddings**. During the training process, BERT learns a set of positional vectors corresponding to token positions up to a maximum length (typically 512). These embeddings are added to the token embeddings to provide position information.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel

# Load pre-trained BERT model
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)

# Position embeddings are stored in the embedding layer as 'position_embeddings.weight'
positional_encodings = model.embeddings.position_embeddings.weight.detach().cpu().numpy()

# Compute cosine similarity between positional encodings
cosine_sim = cosine_similarity(positional_encodings)

# Plot the cosine similarity heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cosine_sim, cmap='viridis', cbar=True, vmin=0)
plt.title('Cosine Similarity between BERT Positional Encoding Vectors')
plt.xlabel('Vector 1')
plt.ylabel('Vector 2')
plt.show()

# Plot the cosine similarity for a specific row
plt.figure(figsize=(6, 4))
plt.plot(cosine_sim[100], label='Cosine similarity of Vector 100')
plt.xlabel('Vector 1')
plt.ylabel('Cosine Similarity')
plt.title('Cosine Similarity for BERT Positional Vector 100')
plt.legend()
plt.show()

## 2. BERT Pre-Training Tasks

BERT is pre-trained using two primary tasks: **Masked Language Modeling (MLM)** and **Next Sentence Prediction (NSP)**. 

#### 1. **Masked Language Modeling (MLM)**

- In the MLM task, a token is replaced with the special `[MASK]` token and the model then tries to predict the original token based on the surrounding context. 
- BERT learns to understand the context and fill in the blanks, which helps it develop contextualized embeddings for each word. 

#### 2. **Next Sentence Prediction (NSP)**

- In addition to MLM, BERT is trained to predict whether one sentence logically follows another. This is called the **Next Sentence Prediction (NSP)** task.
- BERT takes two sentences (or segments) as input and must determine if the second sentence is a continuation of the first.


In [ ]:
from transformers import BertForMaskedLM, BertForNextSentencePrediction
import torch

# Load pre-trained BERT model and tokenizer
model_mlm = BertForMaskedLM.from_pretrained('bert-base-uncased')

sentence = "I want to eat a [MASK]."
inputs = tokenizer(sentence, return_tensors='pt')

with torch.no_grad():
    outputs = model_mlm(**inputs)
    predictions = outputs.logits

masked_index = (inputs['input_ids'] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
top_k = 5
top_k_probabilities, top_k_indices = torch.topk(predictions[0, masked_index, :], top_k, dim=-1)

top_k_probabilities = top_k_probabilities.squeeze().tolist()
top_k_indices = top_k_indices.squeeze().tolist()

top_k_tokens = [tokenizer.decode([idx]) for idx in top_k_indices]

print("\nMasked Language Modeling:")
print("Input tokens:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))
print("\nTop 5 Predictions:")
for token, prob in zip(top_k_tokens, top_k_probabilities):
    print(f"Token: {token}, Probability: {prob:.4f}")

In [ ]:
model_nsp = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

sentence_a = "Sherlock Holmes is my favorite detective"
sentence_b = "I read all the books about him!"
inputs_nsp = tokenizer(sentence_a, sentence_b, return_tensors='pt')

print("Tokens:", tokenizer.convert_ids_to_tokens(inputs_nsp['input_ids'][0]))

# Get NSP predictions
with torch.no_grad():
    outputs_nsp = model_nsp(**inputs_nsp)
    logits = outputs_nsp.logits
    nsp_prediction = torch.argmax(logits).item()

nsp_label = "True" if nsp_prediction == 0 else "False"
print(f"Prediction: {nsp_label}")

In [ ]:
sentence_c = "The stock market saw a significant increase yesterday, surprising many investors."
inputs_nsp_false = tokenizer(sentence_a, sentence_c, return_tensors='pt')

print("Tokens:", tokenizer.convert_ids_to_tokens(inputs_nsp_false['input_ids'][0]))

# Get NSP predictions
with torch.no_grad():
    outputs_nsp_false = model_nsp(**inputs_nsp_false)
    logits_false = outputs_nsp_false.logits
    nsp_prediction_false = torch.argmax(logits_false).item()

nsp_label_false = "True" if nsp_prediction_false == 0 else "False"
print(f"Prediction: {nsp_label_false}")

## 3. Encoder Attention Mechanism

The attention mechanism is a crucial component of BERT’s encoder architecture. It allows the model to weigh the importance of each token in a sequence relative to all other tokens from both the left and right sides, making it **bidirectional**.

#### 1. **Self-Attention Mechanism**
- In BERT, each token in a sequence attends to all other tokens, including itself. This is known as **self-attention**. The goal is to understand the relationship and relevance of each token within the entire sequence.
- The attention mechanism allows BERT to dynamically weigh the relevance of each word in the context of the entire sequence. This helps BERT generate contextualized word representations that take into account the entire sentence.


In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)

sentence = "The dog ate the food because it was hungry"

inputs = tokenizer(sentence, return_tensors='pt')

outputs = model(**inputs)
attention = outputs.attentions  # This returns a list of attention matrices

# Select the attention from the last layer and the first attention head
attention_layer = attention[-1]  # Last layer
attention_head = attention_layer[0, 8].detach().numpy()  # First batch and first head

# Get the tokens for the sentence
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Plot the attention weights
plt.figure(figsize=(12, 8))
sns.heatmap(attention_head, xticklabels=tokens, yticklabels=tokens, cmap='viridis')
plt.xlabel('Tokens')
plt.ylabel('Tokens')
plt.title('Self-Attention Weights (Last Layer, Ninth Head)')
plt.show()

## 4. BERT Fine-Tuning

While BERT is pre-trained on general tasks like Masked Language Modeling (MLM) and Next Sentence Prediction (NSP), its real power comes from **fine-tuning** on specific tasks. Fine-tuning involves training the pre-trained BERT model on a smaller dataset for a particular NLP task such as sentiment analysis, text classification, named entity recognition, or question answering.

The fine-tuning process typically involves:
- Adding a task-specific classification head (a simple dense layer) on top of the pre-trained BERT model.
- Training the entire model (including both the BERT base and the new classification head) on your labeled dataset.
- The fine-tuning process adjusts the weights of the model so that it performs well on the new task, while still retaining the knowledge it gained during pre-training.

In this case, we’ll fine-tune BERT for a sentiment classification task. The model will be trained to classify movie reviews as positive or negative.

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In the Hugging Face `transformers` library, it is possible to provide a custom function for computing evaluation metrics, such as accuracy, by passing the function to the `Trainer` class.

You can pass the `compute_metrics` function as an argument to the `Trainer`. The function will automatically be called during evaluation, and the computed metrics (like accuracy) will be returned as part of the evaluation output.


In [31]:
# Function to compute accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

For the sentiment classification task, we will fine-tune a pre-trained BERT model on the IMDB dataset. The model will be trained to predict whether a movie review is positive or negative based on the text content.

In this snippet, the `IMDB` dataset is loaded using the Hugging Face `datasets` library. This dataset contains movie reviews labeled as **positive** or **negative** for binary sentiment classification. 

In [ ]:
# Load a sentiment analysis dataset
dataset = load_dataset('imdb')
train_dataset = dataset['train'].shuffle(seed=42).select(range(2000))
test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

Before feeding into a pre-trained BERT model, the text data needs to be tokenized and converted into input features. Let's define a `tokenize_function` that will handle this preprocessing step.

In [40]:
# Tokenize the dataset
def tokenize_function(sample):
    return tokenizer(sample['text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In Hugging Face's `transformers` library, it is possible to fine-tune pre-trained models using the **Trainer** class, which abstracts much of the complexity of training deep learning models. The **Trainer** is paired with **TrainingArguments**, which define the parameters for training, evaluation, logging, and saving the model’s progress. These components together allow users to easily fine-tune models for downstream tasks with minimal configuration.

- **Trainer**: The `Trainer` class provides a high-level interface to train, evaluate, and make predictions using a pre-trained model. It handles tasks like optimization, gradient computation, and model checkpointing.

- **TrainingArguments**: This class is used to configure the fine-tuning process. It includes the essential hyperparameters like learning rate, batch size, number of epochs, logging settings...

In [41]:
batch_size = 32
num_train_epochs = 2

learning_rate = 2e-5
weight_decay = 0.01

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every 10 steps
)

# Initialize the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In Hugging Face's `transformers` library, after setting up the **Trainer** and **TrainingArguments**, two key methods are used to manage the model’s training and evaluation:

- **`trainer.train()`**: This method initiates the fine-tuning process of the pre-trained model. It handles the forward and backward passes, computes gradients, and updates the model's parameters using the specified optimization algorithm. The method processes the data in batches, performs any logging and checkpointing as specified in the `TrainingArguments`, and loops over the dataset for the number of epochs defined. In essence, `train()` automates the entire training lifecycle.

- **`trainer.evaluate()`**: After training, this method is used to assess the performance of the model on a validation or test dataset. It runs the model in evaluation mode, meaning no gradients are computed, and it returns metrics such as accuracy, loss, or other custom metrics defined in the `compute_metrics` function. Evaluation is often performed at the end of each epoch during training, but `evaluate()` can also be called manually to test model performance after training.

Together, `train()` and `evaluate()` allow for efficient model fine-tuning, performance tracking, and validation, making it easier to adapt pre-trained models to specific tasks with minimal coding.

In [ ]:
# Fine-tune the model
results = trainer.train()
print(f"Accuracy on the train set: {results.metrics['eval_accuracy']:.2f}")

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(f"Accuracy on the test set: {results['eval_accuracy']:.2f}")